<a href="https://colab.research.google.com/github/bukhtiar-haider/diabetes-prediction/blob/main/Diabetes-Medication-Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from collections import Counter
from sklearn.impute import KNNImputer as KNN
from sklearn.tree import DecisionTreeRegressor
from imblearn.over_sampling import SMOTE
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [ ]:
# Load the data
df = pd.read_csv('P3_Data.csv')
df_test = pd.read_csv('P3_Test.csv')
df.head() # Just taking a quick look at the data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,193.83,UK,49.74,14.16,213.02,343.06,1753.44,-6.03,6.28,8,-5145.92,Low,11.35,1.01,1,68.92,51.16
1,1495.47,Europe,58.02,5.88,137.80,270.48,1665.33,-0.52,9.72,14,11145.82,High,10.30,0.30,5,89.30,0.00
2,1121.31,USA,68.85,9.18,209.54,295.26,3208.65,-4.00,3.86,6,31859.00,Medium,7.98,13.05,4,-11.80,2808.51
3,-305.49,USA,49.29,8.73,171.83,266.06,1704.18,-2.41,4.16,6,8075.76,Low,16.52,2.01,5,41.40,1092.21
4,1457.07,UK,64.98,7.92,74.10,260.22,1922.34,-11.97,5.00,18,9886.72,High,5.37,0.01,1,62.74,0.00


In [ ]:
df_test.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,596.16,Rest,72.24,6.00,154.48,184.18,226.65,-8.21,18.46,10,6547.48,Very high,9.24,0.69,3,45.06,NaN
1,1851.57,Europe,63.09,10.47,200.93,183.48,1101.51,-7.91,3.76,4,-983.62,Medium,8.20,17.90,5,88.64,NaN
2,-945.60,USA,71.22,8.25,190.35,243.96,924.93,-6.35,2.48,6,-2366.10,Very high,11.10,329.72,2,-41.36,NaN
3,1153.38,Europe,62.52,7.41,165.21,245.64,1017.09,-5.84,6.58,14,-503.06,High,7.53,0.00,2,-91.60,NaN
4,-261.63,UK,60.87,10.14,201.96,216.28,1350.96,-2.75,4.02,6,19521.62,High,8.99,360.26,4,154.32,NaN


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F1      1500 non-null   float64
 1   F2      1500 non-null   object 
 2   F3      1500 non-null   float64
 3   F4      1500 non-null   float64
 4   F5      1500 non-null   float64
 5   F6      1500 non-null   float64
 6   F7      1500 non-null   float64
 7   F8      1500 non-null   float64
 8   F9      1500 non-null   float64
 9   F10     1500 non-null   int64  
 10  F11     1500 non-null   float64
 11  F12     1500 non-null   object 
 12  F13     1500 non-null   float64
 13  F14     1500 non-null   float64
 14  F15     1500 non-null   int64  
 15  F16     1500 non-null   float64
 16  Target  1500 non-null   float64
dtypes: float64(13), int64(2), object(2)
memory usage: 199.3+ KB
None


In [ ]:
lis = []
for i in range(0, df.shape[1]):
     
    if(df.iloc[:,i].dtypes == 'object'):
        df.iloc[:,i] = pd.Categorical(df.iloc[:,i])
        #print(marketing_train[[i]])
        df.iloc[:,i] = df.iloc[:,i].cat.codes 
        df.iloc[:,i] = df.iloc[:,i].astype('object')
         
        lis.append(df.columns[i])

lis2 = []
for i in range(0, df_test.shape[1]):
     
    if(df_test.iloc[:,i].dtypes == 'object'):
        df_test.iloc[:,i] = pd.Categorical(df_test.iloc[:,i])
        #print(marketing_train[[i]])
        df_test.iloc[:,i] = df_test.iloc[:,i].cat.codes 
        df_test.iloc[:,i] = df_test.iloc[:,i].astype('object')
         
        lis2.append(df_test.columns[i])

In [ ]:
class OneHotEncoderCategoricalFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, cat_features):
        self.cat_features = cat_features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.get_dummies(X, columns=self.cat_features)

In [ ]:
cat_features = ['F2', 'F12']

encoder = OneHotEncoderCategoricalFeatures(cat_features)
df = encoder.transform(df)

In [ ]:
df_test = encoder.transform(df_test)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F1      1500 non-null   float64
 1   F3      1500 non-null   float64
 2   F4      1500 non-null   float64
 3   F5      1500 non-null   float64
 4   F6      1500 non-null   float64
 5   F7      1500 non-null   float64
 6   F8      1500 non-null   float64
 7   F9      1500 non-null   float64
 8   F10     1500 non-null   int64  
 9   F11     1500 non-null   float64
 10  F13     1500 non-null   float64
 11  F14     1500 non-null   float64
 12  F15     1500 non-null   int64  
 13  F16     1500 non-null   float64
 14  Target  0 non-null      float64
 15  F2_0    1500 non-null   uint8  
 16  F2_1    1500 non-null   uint8  
 17  F2_2    1500 non-null   uint8  
 18  F2_3    1500 non-null   uint8  
 19  F12_0   1500 non-null   uint8  
 20  F12_1   1500 non-null   uint8  
 21  F12_2   1500 non-null   uint8  
 22  

In [ ]:
df_nan_count = df.isna().sum().sum()
print("NaN count in Training DF: ", df_nan_count)

df_nan_count = df_test.isna().sum().sum()
print("NaN count in Holdout DF: ", df_nan_count)
#1500 NaN from Target column being empty

NaN count in Training DF:  0
NaN count in Holdout DF:  1500


In [ ]:
conditions = [
    (df['Target'] == 0),
    (df['Target'] != 0)
]
values = [False, True]
y_ = np.select(conditions, values)

In [ ]:
y = df['Target']
X = df.drop('Target', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y_, random_state=0)
scaler_x = StandardScaler()
X_holdout = df_test.drop('Target', axis = 1)
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.fit_transform(X_test)
X_holdout = scaler_x.transform(X_holdout)

In [ ]:
a = Counter(y_test == 0)
print(a[1]/a[0])
b = Counter(y_train == 0)
print(b[1]/b[0])

0.42585551330798477
0.4240506329113924


In [ ]:
model = LinearRegression()
params = {'fit_intercept':[True, False], 'copy_X':[True,False]}
clf = GridSearchCV(model, params)
clf.fit(X_train, y_train)
clf.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_copy_X', 'param_fit_intercept', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [ ]:
clf.cv_results_['mean_test_score']
#0.87454038

array([0.77882197, 0.03170644, 0.77882197, 0.03170644])

In [ ]:
clf.cv_results_['std_test_score']

array([0.03116439, 0.09378709, 0.03116439, 0.09378709])

In [ ]:
results = []
names = []
testing = []

**LINEAR REGRESSION**

In [ ]:
lr = LinearRegression()

cv_results = cross_validate(lr, X_train, y_train, scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'], cv=10)
print(cv_results['test_neg_root_mean_squared_error'].mean())
print(cv_results['test_neg_mean_squared_error'].mean())
#results.append(cv_results['test_score'])
#names.append('DT')

-506.93498400039516
-258556.895422936


In [ ]:
print(cv_results['test_neg_root_mean_squared_error'])
print(cv_results['test_neg_mean_squared_error'])

[-508.22627705 -517.3029808  -462.30512437 -522.21229459 -516.62664964]
[-258293.94868366 -267602.3739454  -213726.02801467 -272705.6806192
 -266903.09512011]


In [ ]:
lr.fit(X_train, y_train)
lr_res = lr.predict(X_test)
lr_testing_rmse = mean_squared_error(lr_res, y_test, squared=False)
lr_testing_mse = mean_squared_error(lr_res, y_test, squared=True)
print(lr_testing_rmse)
print(lr_testing_mse)
#testing.append(lr_testing)

1674475000373079.5
2.8038665268744244e+30


**SGD**

In [ ]:
sgd = SGDRegressor()

cv_results = cross_validate(sgd, X_train, y_train, scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'], cv=10)
print(cv_results['test_neg_root_mean_squared_error'].mean())
print(cv_results['test_neg_mean_squared_error'].mean())
#results.append(cv_results['test_score'])
#names.append('DT')

-505.40169110891173
-256992.084836854


In [ ]:
print(cv_results['test_neg_root_mean_squared_error'])
print(cv_results['test_neg_mean_squared_error'])

[-518.01385009 -509.65302671 -581.53558293 -458.02739    -486.20884128
 -429.94572555 -538.31904083 -502.16526436 -510.01371236 -520.13447698]
[-268338.3488806  -259746.20763644 -338183.63421466 -209789.08999211
 -236399.03733635 -184853.32691783 -289787.38971586 -252169.95273427
 -260113.98679609 -270539.87414434]


In [ ]:
sgd.fit(X_train, y_train)
sgd_res = sgd.predict(X_test)
sgd_testing_rmse = mean_squared_error(sgd_res, y_test, squared=False)
sgd_testing_mse = mean_squared_error(sgd_res, y_test, squared=True)
print(sgd_testing_rmse)
print(sgd_testing_mse)
#testing.append(lr_testing)

509.895720744063
259993.64603310745


**ADABOOST**

In [ ]:
adb = AdaBoostRegressor(n_estimators=100, base_estimator = GradientBoostingRegressor(n_estimators = 500, subsample = 0.6, random_state = 0), random_state=0)

cv_results = cross_validate(adb, X_train, y_train, scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'], cv=10)
print(cv_results['test_neg_root_mean_squared_error'].mean())
print(cv_results['test_neg_mean_squared_error'].mean())
#results.append(cv_results['test_score'])
#names.append('DT')

-428.7446718972721
-185035.22813652415


In [ ]:
print(cv_results['test_neg_root_mean_squared_error'])
print(cv_results['test_neg_mean_squared_error'])

[-455.58731646 -406.5502985  -456.6737127  -415.70310177 -409.08532955
 -352.13400254 -483.19719477 -447.71759381 -444.2357267  -416.56244218]
[-207559.80292375 -165283.14520804 -208550.87986806 -172809.06882289
 -167350.80685241 -123998.3557472  -233479.52903227 -200451.04380394
 -197345.38087286 -173524.26823382]


In [ ]:
adb.fit(X_train, y_train)
adb_res = adb.predict(X_test)
adb_testing_rmse = mean_squared_error(adb_res, y_test, squared=False)
adb_testing_mse = mean_squared_error(adb_res, y_test, squared=True)
print(adb_testing_rmse)
print(adb_testing_mse)
#testing.append(lr_testing)

420.24426842318445
176605.24514253752


**MLP**

In [ ]:
mlp = MLPRegressor(max_iter = 5000, hidden_layer_sizes=(60,120,120,60,30), solver='lbfgs', alpha=0.75, random_state=0)

cv_results = cross_validate(mlp, X_train, y_train, scoring=['r2', 'neg_mean_squared_error'], cv=10)
print("MSE", cv_results['test_neg_mean_squared_error'].mean())
print("R2", cv_results['test_r2'].mean())
#results.append(cv_results['test_score'])
#names.append('MLP')
#MSE -2816.863770453625
#R2 0.997568859496638

In [ ]:
print("MSE", cv_results['test_neg_mean_squared_error'])
print("R2", cv_results['test_r2'])
#MSE [-1450.45159792 -2035.03274576 -2630.47819955 -4571.67275942
# -4212.38273133 -2932.43785576 -3341.06100291 -2826.1061419
# -2320.00551234 -1686.57085587]
#R2 [0.99895997 0.99804791 0.99741943 0.99672696 0.99651369 0.99673755
# 0.997764   0.99785467 0.99779528 0.99800384]

MSE [-1450.45159792 -2035.03274576 -2630.47819955 -4571.67275942
 -4212.38273133 -2932.43785576 -3341.06100291 -2826.1061419
 -2320.00551234 -1686.57085587]
R2 [0.99895997 0.99804791 0.99741943 0.99672696 0.99651369 0.99673755
 0.997764   0.99785467 0.99779528 0.99800384]


In [ ]:
mlp = MLPRegressor(max_iter = 5000, hidden_layer_sizes=(60,120,120,60,30), solver='lbfgs', alpha=0.75, random_state=0)

mlp.fit(X_train, y_train)
mlp_res = mlp.predict(X_test)
mlp_testing_r2 = r2_score(mlp_res, y_test)
mlp_testing_r2
#testing.append(mlp_testing)
#0.990743234357331


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.990743234357331

In [ ]:
mlp_testing_mse = mean_squared_error(mlp_res, y_test, squared=True)
mlp_testing_mse
#10977.18549070397


10977.18549070397

In [ ]:
predictions = mlp.predict(X_holdout)
df_test['Target'] = predictions.astype(float)
df_test['Target']

0        369.010011
1         -0.267030
2       2419.514388
3        381.815941
4          1.725393
           ...     
1495     557.235925
1496     257.572436
1497     572.938840
1498    2482.617207
1499     154.496769
Name: Target, Length: 1500, dtype: float64

In [ ]:
df_test.to_csv("CE802_P2_Test.csv", index = False)